# 🚀 02_05_LandingGear_Data_Generation.ipynb

This notebook generates **synthetic landing gear system data** for the Databricks AeroDemo pipeline.
It creates component-level metrics focused on the landing gear performance and integrity of the aircraft fleet.

---

### 📋 What this notebook does:

✅ **Sets up aircraft fleet**
- Uses the same fleet IDs as earlier generators (`A320`, `B737`, `A330`)

✅ **Generates daily landing gear records**
- For each aircraft:
  - Simulates **extension/retraction cycles** and **hydraulic pressure**
  - Includes **shock absorber status**, **brake temperature**, and **gear alignment**
  - Adds occasional anomalies for realistic failure or warning cases

✅ **Creates a structured dataset**
- Component records → CSV for Auto Loader (`/Volumes/arao/aerodemo/tmp/landing_gear`)

✅ **Saves output with timestamped filenames**
- Ensures multiple runs create separate files for easy ingestion

---

### 🛠 Key points:
- **Focus area:** Landing gear operation and safety monitoring
- **Data diversity:** Includes normal cycles, wear indicators, and anomaly events
- **Downstream integration:** Feeds DLT tables like `twin_landing_gear` and risk scoring

---

### 🔗 Where this fits:
This notebook is part of the **02_ series** synthetic data generators:
- `02_01_Sensor_Data_Generation.ipynb`
- `02_02_Engine_Data_Generation.ipynb`
- `02_03_CabinPressurization_Data_Generation.ipynb`
- `02_04_Airframe_Synthetic_Data_Generation.ipynb`
- `02_05_LandingGear_Data_Generation.ipynb` (this file)
- Future: additional components like avionics, electrical [systems](url)

In [0]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

def generate_landing_gear_data(num_records_per_aircraft=100):
    aircraft_ids = ["A320_101", "A330_201", "B737_301"]
    data = {
        'landing_gear_id': [],
        'aircraft_id': [],
        'event_timestamp': [],
        'extension_cycles': [],
        'hydraulic_pressure': [],
        'shock_absorber_status': [],
        'brake_temperature': [],
        'gear_alignment': []
    }

    for aircraft_id in aircraft_ids:
        for i in range(num_records_per_aircraft):
            random_days_ago = random.randint(0, 6)
            random_time = datetime.now() - timedelta(
                days=random_days_ago,
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59)
            )
            data['landing_gear_id'].append(f'LG_{aircraft_id}_{i:03d}')
            data['aircraft_id'].append(aircraft_id)
            data['event_timestamp'].append(random_time.strftime("%Y-%m-%d %H:%M:%S"))
            data['extension_cycles'].append(random.randint(0, 10))
            data['hydraulic_pressure'].append(round(np.random.uniform(2000, 3000), 2))  # psi
            data['shock_absorber_status'].append(round(np.random.uniform(0.8, 1.0), 2))  # normalized
            data['brake_temperature'].append(round(np.random.uniform(100, 500), 2))  # °C
            data['gear_alignment'].append(round(np.random.uniform(-2.0, 2.0), 2))  # degrees

    df = pd.DataFrame(data)
    return df

# Generate DataFrame
df = generate_landing_gear_data()

# Save to Auto Loader-compatible path
output_path = "/Volumes/arao/aerodemo/tmp/landing_gear"
os.makedirs(output_path, exist_ok=True)
output_file = f"{output_path}/landing_gear_sample.csv"
if os.path.exists(output_file):
    os.remove(output_file)  # safely remove old file

df.to_csv(output_file, index=False)

print(f"✅ Landing gear data generated: {len(df)} rows saved to {output_file}")